# GANによる画像生成

commit用

```mycode/5_gan_generation/```

In [1]:
!gpustat

/bin/sh: gpustat: command not found


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
import random
import math
import time
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms

## 5-3 Self-Attention GANの概要

2018年時点で最高峰のGANの１つである**BigGAN**もSAGANをベースとしているGANである．本節では

- Self-Attention
- pointwise convolution
- Spectral Normalization

の３つのテーマについての理解を目標とする．

### 従来のGANの問題点
従来のGANの転置畳み込みを繰り返す操作は特徴量マップが大きくなりますが，転置畳み込みの繰り返しでは**局所的な情報の拡大にしかならない**問題がある．

よってより良い画像生成を実現するためには可能であれば拡大する際に画像全体の大域的な情報を考慮する仕組みが必要になる．

### Self-Attentionの導入
従来のニューラルネットワークでは前の層の出力がそのまま次の層に入力されるので式としては
$$
y = x
$$
で表される．しかしこれでは局所的な性質が常に伝わっていくので，さらに大域的な情報を用いて表される$o$を導入する．この$o$に係数$\gamma$をかけた値を足して次の層に伝達することを考える．
$$
y = x +\gamma o
$$
この$o$をどのように求めるかについては以下の手順で求める．
1. $x$を[C,W,H]から[C,N]に変形する. $N=W\times H$
2. $S=x^Tx$ という行列を用意する．これは画像位置$i$と画像位置$j$の関係性を表す行列である．
3. $S$を行方向にソフトマックス関数にかける．これで画像位置$i$と画像位置$j$の関係性の総和が1になり扱いやすくなる．これを$\beta$とし，これを**Attention Map**という．
4. 上で作った$\beta$を$x$にかけることで$o$を得る．つまり$o = x\beta^T$である．$o=[C,N]$

この$o = x\beta^T$について
$$
o_{c=k,n=j} = \sum_{i=1}^Nx_{ki}\beta_{ji}
$$
この式はチャネル$k$において位置$j$について位置$i=1,2,\cdots N$の全ての影響を総和することを意味している．
$x_{ki}$ はチャネル$k$の位置$i$の特徴量を表していて，$\beta_{ji}$は位置$j$と位置$i$の関係性を表して，この$\sum$はそれらの積の総和を求めている．


In [4]:
#　最初のサイズ変形 (B,C,W,H)  to (B,C,N)
X = torch.randn(3,32,64,64)
X = X.view(X.shape[0], X.shape[1], -1)

# 掛け算
X_T = X.permute(0,2,1)
S = torch.bmm(X_T, X)

# 規格化
m = nn.Softmax(dim=-2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0,2,1)

# Self-Attention Map
o = torch.bmm(X, attention_map.permute(0,2,1))

### 1×1 Convolutions (pointwise convolution)
Self-Attention の制限は非常に強いのでこのままでは学習はうまくいかないことが多い．
そこでpointwise convolutionと呼ばれる前処理をした特徴量マップをSelf-Attentionに渡してあげることによって学習がうまくいくようにする．
またこれをすることによって特徴量マップのチャネル数を変えることができる(一般にはチャネル数を減らす)ので計算量を調整することができる．

またpointwise convolutionでは次の言葉が使われる
- query：元の入力$x$の転置に対応するもの
- key：元の入力$x$に対応するもの
- value：Attention Mapと掛け算する対象

出力のチャネルはカーネルの数となる．カーネルのチャネルは入力のチャネルに合わせる

In [5]:
X = torch.randn(3,32,64,64)

# 1×1の畳み込み層によるpointwise convolutionを用意　
query_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)

key_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1]//8, kernel_size=1)

value_conv = nn.Conv2d(
    in_channels=X.shape[1], out_channels=X.shape[1], kernel_size=1)

# 畳み込みをしてからサイズを変更する
proj_query = query_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])
proj_query = proj_query.permute(0,2,1)
proj_key = key_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])

# 掛け算
S = torch.bmm(proj_query, proj_key) # bmmはバッチごとの掛け算を行う

# 規格化
m = nn.Softmax(dim=-2)
attention_map_T = m(S)
attention_map = attention_map_T.permute(0,2,1)

# Self-Attention Map
proj_value = value_conv(X).view(
    X.shape[0], -1, X.shape[2]*X.shape[3])
o = torch.bmm(proj_value, attention_map.permute(0,2,1))

### Spectral Normalization
- Spectral Normalization：畳み込みの重みに対する規格化
- Batch Normalization：ディープラーニングモデルを流れるデータに対する規格化

***リプシッツ連続性(Lipschitz continuity)：***判別機Dの出力が入力データに対して頑健性を持つこと
具体的には層を表す行列の固有値のうち最大の固有値で全ての重みを割ることで規格化している．固有値は固有ベクトルの拡大率なので，一番拡大される固有値で全てを規格化することによって流れるデータが想定外に大きくなることを防ぐ．

In [6]:
z_dim = 20
image_size = 64
nn.utils.spectral_norm(nn.ConvTranspose2d(
    z_dim, image_size*8, kernel_size=4,stride=1))

ConvTranspose2d(20, 512, kernel_size=(4, 4), stride=(1, 1))

## 5-4 Self-Attention GANの学習，生成の実装

### Self-Attentionモジュールの実装
$$
y = x + \gamma o
$$
$\gamma$ は初期値０から始めるが，学習させる対象なのでこれも```nn.Parameter()```を用いて実装する．

Attention MapはあとでAttentionの強さを可視化するために出力させてる

In [15]:
class Self_Attention(nn.Module):
    """ Self-AttentionのLayer"""
    
    def __init__(self, in_dim):
        super(Self_Attention, self).__init__()
        
        # 1×1の畳み込み層によるpointwise convolutionを用意　
        self.query_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)

        self.key_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)

        self.value_conv = nn.Conv2d(
            in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        
        # 規格化
        self.softmax = nn.Softmax(dim=-2)
        
        # gamma
        # 学習させるのでnn.Parameter　を用いる
        self.gamma = nn.Parameter(torch.zero(1))
        
        
    def forward(self, x):
        
        # 入力変数
        X = x
        
        # 畳み込みをしてからサイズを変更する
        proj_query = self.query_conv(X).view(X.shape[0], -1, X.shape[2]*X.shape[3])
        proj_query = self.proj_query.permute(0,2,1)
        proj_key = self.key_conv(X).view(X.shape[0], -1, X.shape[2]*X.shape[3])
        
        
        # 掛け算
        S = torch.bmm(proj_query, proj_key) # bmmはバッチごとの掛け算を行う  
        
        # 規格化
        attention_map_T = self.softmax(S)
        attention_map = attention_map_T.permute(0,2,1)
        
        # Self-Attention Map
        proj_value = self.value_conv(X).view(X.shape[0], -1, X.shape[2]*X.shape[3])
        o = torch.bmm(proj_value, attention_map.permute(0,2,1))
        
        # Self-Attention　MapのテンソルサイズをXに揃えて出力
        o = o.view(X.shape[0],X.shape[1],X.shape[2],X.shape[3])
        out = x + self.gamma*o
        
        return out , attention_map

### Generatorの実装
DCGANとの変更点は以下の2点である．
1. 最後のlaseのLayer以外には転置畳み込み層にSpectral Normalizationを追加する．
2. layer3-layer4と，layer4-lastの二箇所にSelf-Attentionモジュールを追加する

In [21]:
class Generator(nn.Module):
    
    def __init__(self, z_dim = 20, image_size=64):
        super(Generator, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            #　ここでConvTranspose2dにspectral_norm を作用させる
#             nn.ConvTranspose2d(z_dim, image_size*8, kernel_size=4, stride=1),
            nn.utils.spectral_norm(nn.ConvTranspose2d(z_dim, image_size*8, kernel_size=4,stride=1)),
            nn.BatchNorm2d(image_size*8),
            nn.ReLU(inplace=True))
        
        self.layer2 = nn.Sequential(
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size*8, image_size*4, kernel_size=4,stride=2, padding=1)), #spectral_norm
            nn.BatchNorm2d(image_size*4),
            nn.ReLU(inplace=True))
        
        self.layer3 = nn.Sequential(
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size*4, image_size*2, kernel_size=4,stride=2, padding=1)), #spectral_norm
            nn.BatchNorm2d(image_size*2),
            nn.ReLU(inplace=True))
        
        # Self-Attention 層
        self.self_attention1 = Self_Attention(in_dim=image_size*2)
              
        self.layer4 = nn.Sequential(
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size*2, image_size, kernel_size=4,stride=2, padding=1)), #spectral_norm
            nn.BatchNorm2d(image_size),
            nn.ReLU(inplace=True))

        # Self-Attention 層
        self.self_attention2 = Self_Attention(in_dim=image_size)
        
        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh())
        # 白黒なので出力は1チャネル

    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out, attention_map1 = self.self_attention1(out)
        out = self.layer4(out)
        out, attention_map2 = self.self_attention2(out)
        out = self.last(out)
        
        return out, attention_map1, attention_map2

### Discriminatorの実装
DCGANからの変更点は以下の2点
1. last以外のlayerに畳み込み層にSpectral Normalizationを追加する
2. layer3-layer4と，layer4-lastの二箇所にSelf-Attentionモジュールを追加する

In [23]:
class Discriminator(nn.Module):
    
    def __init__(self, z_dim=20, image_size=64):
        super(Discriminator, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(1, image_size, kernel_size=4, stride=2, padding=1)), # Spectral Normalizationの追加
            nn.LeakyReLU(0.1, inplace=True))

        self.layer2 = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(image_size, image_size*2, kernel_size=4, stride=2, padding=1)),# Spectral Normalizationの追加
            nn.LeakyReLU(0.1, inplace=True))

        self.layer3 = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(image_size*2, image_size*4, kernel_size=4, stride=2, padding=1)),# Spectral Normalizationの追加
            nn.LeakyReLU(0.1, inplace=True))
        
        # Self-Attention層の追加
        self.self_attention1 = Self_Attention(in_dim=image_size*4)

        self.layer4 = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(image_size*4, image_size*8, kernel_size=4, stride=2, padding=1)),# Spectral Normalizationの追加
            nn.LeakyReLU(0.1, inplace=True))

        # Self-Attention層の追加
        self.self_attention2 = Self_Attention(in_dim=image_size*8)
        
        self.last = nn.Conv2d(image_size*8, 1, kernel_size=4, stride=1)
        
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out, attention_map1 = self.self_attention1(out)
        out = self.layer4(out)
        out, attention_map2 = self.self_attention1(out)
        out = self.last(out)
        
        return out, attention_map1, attention_map2

### DataLoaderの実装
DCGANと同じ

In [24]:
def make_datapath_list():
    
    train_img_list = list()
    
    for img_idx in range(200):
        img_path = "./data/img_78/img_7_" + str(img_idx) + ".jpg"
        train_img_list.append(img_path)
        
        img_path = "./data/img_78/img_8_" + str(img_idx) + ".jpg"
        train_img_list.append(img_path)
        
    return train_img_list


class ImageTransform():
    """
    画像クラスの前処理
    """
    def __init__(self, mean, std):
        self.data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
    
    def __call__(self, img):
        return self.data_transform(img)
    

class GAN_Img_Dataset(data.Dataset):
    """
    画像のDataset
    """
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform
            
    def __len__(self):
        """画像の枚数を返す"""
        return len(self.file_list)
    
    def __getitem__(self,index):
        """前処理をした画像のTensor形式のデータを取得"""
        
        img_path = self.file_list[index]
        img = Image.open(img_path) #H,W,C
        
        # 前処理
        img_transformed = self.transform(img)
        
        return img_transformed


In [26]:
# DataLoaderの作成
train_img_list = make_datapath_list()
# print(train_img_list)

# Datasetの作成
mean = (0.5,)
std = (0.5,)
train_dataset = GAN_Img_Dataset(train_img_list, ImageTransform(mean, std))

# DataLoaderの作成
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                              batch_size = batch_size, shuffle=True)

### ネットワークの初期化と学習の実施
SAGANの損失関数はhinge version of the adversarial lossと呼ばれるものに変更している
$$
-\frac{1}{M}\sum_{i=1}^M[l_i \ast \min (0, -1+y_i) + (1-l_i)\ast \min(0, -1-y_i)]
$$
これは実装では
```d_loss_real = torch.nn.ReLU()(1.0-d_out_real).mean()``` となる．ここでReLUは活性化関数ではなくmin(正確にはmax)の部分に相当する．
**シグモイドにはしてないがこれは？**

またGの方では以下のように損失関数を改める．
$$
-\frac{1}{M}\sum_{i=1}^M D(G(z_i))
$$
これによって実装は
```g_loss = - d_out_fake.mean()``` となる．

このような損失関数が用いられているのは経験的にうまくいくからである．